In [27]:
import requests
from bs4 import BeautifulSoup
from dataclasses import dataclass, asdict
import pycountry
from datetime import date
import json

@dataclass
class Member:
    region: str
    iso_code_3: str
    iso_code_2: str
    participating: bool
    name_en: str

opengov_members: list[Member] = []

url = "https://www.opengovpartnership.org/members"

response = requests.get(url, timeout=10)

if response.status_code < 200 or response.status_code > 300:
    raise Exception(f"Wrong status code: {response.status_code}")

soup = BeautifulSoup(response.content, "html.parser")

member_tags =  soup.find_all("li", class_="js-member")

country_mapping = {
    c.alpha_3: c.alpha_2 for c in pycountry.countries
}

for member_tag in member_tags:
    iso_code_3 = member_tag.get("data-code") 
    opengov_members.append(Member(
        region=member_tag.get("data-region"),
        iso_code_3=iso_code_3,
        iso_code_2=country_mapping.get(iso_code_3),
        participating= member_tag.get("data-participating", "").strip() == "active",
        name_en=member_tag.text.strip()
    ))

with open(f"{date.today().strftime("%Y-%m-%d")}-opengov-members.json", "w") as f:
    json.dump([asdict(m) for m in opengov_members], f, indent=2)